In [1]:
import pandas as pd
import unicodedata

In [2]:
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [3]:
populacao = pd.read_csv("../data/populacao/populacao.csv")

In [4]:
populacao.Cidade = populacao.Cidade.apply(lambda x: x[:-5])

In [5]:
regioes = pd.read_csv("../data/geoloc/regioes-saude.csv")

In [6]:
regioes.municipio = regioes.municipio.apply(lambda x: remove_accents(x).lower().strip())

In [7]:
regioes.set_index("municipio",inplace=True)

In [8]:
def get_ibge_code(cityname): 
    try:
        return regioes.loc[remove_accents(cityname).lower().replace("-", " "),"CODIBGE"]
    except KeyError:
        print(f"Erro com {cityname}")
        return None

In [9]:
populacao["COD_IBGE"] = populacao.Cidade.apply(lambda x: get_ibge_code(x)) 

In [13]:
populacao.set_index("COD_IBGE",inplace=True)

In [14]:
populacao.to_csv("../parsed-data/populacao.csv")

Teste

In [17]:
doenca = pd.read_csv( "../parsed-data/doencas.csv")

In [39]:
serie =  doenca.query("ano == 2020").set_index("Ibge")["Hipertensão"]

In [49]:
a = serie.div(populacao["2020"])

In [52]:
pd.DataFrame({"IBGE": a.index.values, "indeice": a.values})

,IBGE,indeice
0,420005,0.239403
1,420010,0.041314
2,420020,0.349859
3,420030,0.174079
4,420040,0.039569
...,...,...
290,421950,0.073719
291,421960,0.411222
292,421970,0.079426
293,421985,0.039141
